# **Programming Assessment \#4**

Names: \<please supply your names\>

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [88]:
# The code is defining a function called `make_p` that takes a DataFrame `df` as input.
# lang must contain a column 'count' that contains number types
#load error data
import pandas as pd
%pip install editdistpy
from editdistpy import damerau_osa as ld
def make_p(df):
    total = df['count'].sum()
    inv = 1/total
    df['P'] = df['count']*inv
    print (df['P'])
    return df


In [66]:
x=''
with open("count_1edit.txt", "r") as f:
    x = f.read().splitlines()
gen = (dat.split('\t') for dat in x)

In [67]:
data = [(rec[0],int(rec[1]))for rec in gen]
err = pd.DataFrame.from_records(data, columns=['error', 'count'])
err = make_p(err)
err = err.set_index(['error'])
print(err)

0       0.023469
1       0.021908
2       0.019732
3       0.019169
4       0.014307
          ...   
1582    0.000026
1583    0.000026
1584    0.000026
1585    0.000026
1586    0.000026
Name: P, Length: 1587, dtype: float64
       count         P
error                 
e|i      917  0.023469
a|e      856  0.021908
i|e      771  0.019732
e|a      749  0.019169
a|i      559  0.014307
...      ...       ...
 |c        1  0.000026
 |a        1  0.000026
 |'        1  0.000026
 w|        1  0.000026
 t|t       1  0.000026

[1587 rows x 2 columns]


In [23]:
#loading corpus files
import nltk
from nltk.corpus import gutenberg
nltk.download('gutenburg')
# nltk.corpus.gutenberg.fileids()

[nltk_data] Error loading gutenburg: Package 'gutenburg' not found in
[nltk_data]     index


False

In [58]:
from collections import Counter
# TODO: consolidate into a Finder class
import string
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename) if word not in string.punctuation]
  all_words.update(words)
    

Extracting all documents from NLTK's Project Gutenberg Collection...
          index     0
0          emma   866
1            by  8512
2          jane   303
3        austen     3
4          1816     1
...         ...   ...
42307  endowing     1
42308   delving     1
42309  germinal     1
42310   blither     1
42311  ushering     1

[42312 rows x 2 columns]


In [61]:
lang = pd.DataFrame.from_dict(all_words, orient='index').reset_index()
print(lang)

          index     0
0          emma   866
1            by  8512
2          jane   303
3        austen     3
4          1816     1
...         ...   ...
42307  endowing     1
42308   delving     1
42309  germinal     1
42310   blither     1
42311  ushering     1

[42312 rows x 2 columns]


In [62]:
lang = lang.rename(columns={'index':'word', 0:'count'})
lang = lang.set_index(['word'])
print(lang)

          count
word           
emma        866
by         8512
jane        303
austen        3
1816          1
...         ...
endowing      1
delving       1
germinal      1
blither       1
ushering      1

[42312 rows x 1 columns]


In [68]:
lang = make_p(lang)
print(lang)

word
emma        3.892499e-04
by          3.825976e-03
jane        1.361925e-04
austen      1.348441e-06
1816        4.494802e-07
                ...     
endowing    4.494802e-07
delving     4.494802e-07
germinal    4.494802e-07
blither     4.494802e-07
ushering    4.494802e-07
Name: P, Length: 42312, dtype: float64
          count             P
word                         
emma        866  3.892499e-04
by         8512  3.825976e-03
jane        303  1.361925e-04
austen        3  1.348441e-06
1816          1  4.494802e-07
...         ...           ...
endowing      1  4.494802e-07
delving       1  4.494802e-07
germinal      1  4.494802e-07
blither       1  4.494802e-07
ushering      1  4.494802e-07

[42312 rows x 2 columns]


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*

In [75]:
# import numpy as np
# def get_d(str_1, str_2):
#     rows = len(str_1) + 1
#     cols = len(str_2) + 1
#     dMatrix = np.zeros((rows, cols), dtype=int)
#     dMatrix[0]= [*range(0, cols)]
#     for i in range(1, rows):
#         dMatrix[i][0] = i
            
        
#     for col in range(1, cols):
#         for row in range(1, rows):
#             cost = 0
#             if str_1[row - 1] == str_2[col - 1]:
#                cost = -1
#             dMatrix[row][col] = min(dMatrix[row - 1][col], dMatrix[row][col - 1], dMatrix[row-1][col-1] + cost)+1
#             if row > 1 and col > 1:
#                 dMatrix[row][col] = min (dMatrix[row-2][col-2],dMatrix[row][col])
    
#     return dMatrix[row][col]



In [95]:

def find_err(str_1, str_2):
    rows = len(str_1) + 1
    cols = len(str_2) + 1
    dMatrix = np.zeros((rows, cols), dtype=int)
    dMatrix[0]= [*range(0, cols)]
    for i in range(1, rows):
        dMatrix[i][0] = i
            
        
    for col in range(1, cols):
        for row in range(1, rows):
            if str_1[row - 1] == str_2[col - 1]:
               cost = 0
            else:
               cost = 1
            if str_1[row - 1] == str_2[col] and str_1[row] == str_2[col-1]:
                # transpose
                pass
            dMatrix[row][col] = min(dMatrix[row - 1][col] + 1, dMatrix[row][col - 1] + 1, dMatrix[row-1][col-1] + cost)
            if row > 1 and col > 1:
                dMatrix[row][col] = min (dMatrix[row-2][col-2],dMatrix[row][col])
    
    return dMatrix[row][col]


def candidates(input, all_words):
      # 1 edit distance away from word
      if input in all_words: return [input]
      # candidates = [w for w in all_words if get_d(input,w) <3 ]

      candidates = None
      d = 1
      MAX = 90 #magic number set to the longest possible edit distance ig?
      while not candidates and d <= MAX:
            candidates = [w for w in all_words if ld.distance(input,w,d) > -1]
            d+=1

      return candidates
def spell_correct(candidate_list,tok):
    if tok in candidate_list: return tok
    n=10
    probable = lang.loc[candidate_list].sort_values(['P'], ascending=False).head(n)
    print(lang.loc[candidate_list])
      # maxI = 0
      # for i in range(len(candidate_list)):
      #       x = P(candidate_list[i], all_words)
      #       if x > maxX:
      #             maxX = x
      #             maxI = i
      # TODO: Add the error model. Kyle, can you please make an algo that finds the first error and query the probability?
    return probable.index

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
#Kyle's error trace
# def calculate_edit_type_and_edit(input_word, candidate_word):
#     if len(input_word) == len(candidate_word):
#         # Check for substitution
#         diff_count = 0
#         edit_type = "sub"
#         edit = ""
#         for i in range(len(input_word)):
#             if input_word[i] != candidate_word[i]:
#                 diff_count += 1
#                 edit += candidate_word[i]
#         if diff_count == 1:
#             return edit_type, edit

#     elif len(input_word) + 1 == len(candidate_word):
#         # Check for insertion
#         edit_type = "ins"
#         edit = ""
#         i, j = 0, 0
#         while i < len(input_word) and j < len(candidate_word):
#             if input_word[i] != candidate_word[j]:
#                 edit += candidate_word[j]
#                 j += 1
#             else:
#                 i += 1
#                 j += 1
#         if j == len(candidate_word):
#             return edit_type, edit

#     elif len(input_word) - 1 == len(candidate_word):
#         # Check for deletion
#         edit_type = "del"
#         edit = ""
#         i, j = 0, 0
#         while i < len(input_word) and j < len(candidate_word):
#             if input_word[i] != candidate_word[j]:
#                 edit += input_word[i]
#                 i += 1
#             else:
#                 i += 1
#                 j += 1
#         if i == len(input_word):
#             return edit_type, edit

#     # Check for transposition
#     if len(input_word) == len(candidate_word) and input_word != candidate_word:
#         for i in range(len(input_word) - 1):
#             if input_word[i] == candidate_word[i + 1] and input_word[i + 1] == candidate_word[i]:
#                 edit_type = "trans"
#                 edit = input_word[i:i+2]
#                 return edit_type, edit

#     return None, None  # No valid edit type found

# # Get user input
# #user_input = input("Enter a word for spell correction: ")

# corpus = "This is a sample text for the spell correction model. Modeler is working fine."
# model = Modeler(corpus)

# print(model.model)

# finder = Finder(model.model)

# token = "food"
# candidates = finder.getCandidates(token)

# print(candidates)

# results = []
# valid_edit_types = ["ins", "trans", "sub" ,"del", ]
# for candidate in candidates:
#     candidate_word, probability = candidate[0], candidate[1]
#     edit_type, edit = calculate_edit_type_and_edit(token, candidate_word)
#     print("\n")
#     print(edit_type,edit)
#     print("\n")
#     if edit_type in valid_edit_types:
#         result = {
#             "word": token,
#             "candidate": candidate_word,
#             "edit_type": edit_type,
#             "edit": edit,
#             "P(c)": probability,
#             "P(w|c)": model.model[candidate_word],
#             "P(c) x P(w|c)": probability * model.model[candidate_word]
#         }
#         results.append(result)
#     else:
#         result = "broken"
#         results.append(result)

# df = pd.DataFrame(results)

# print(df)


In [98]:
#main function
test_input = "thexxxxyyz"
c = candidates(test_input, all_words)
spell_correct(c,test_input)

          count         P
word                     
they      13104  0.005890
theory       25  0.000011
thereby      47  0.000021
thebez        3  0.000001
theology      5  0.000002


,count,P
word,,
they,13104,0.005890
thereby,47,0.000021
theory,25,0.000011
theology,5,0.000002
thebez,3,0.000001
